# HateXplain

https://huggingface.co/Hate-speech-CNERG/bert-base-uncased-hatexplain-rationale-two


The model is used for classifying a text as Abusive (Hatespeech and Offensive) or Normal. The model is trained using data from Gab and Twitter and Human Rationales were included as part of the training data to boost the performance. The model also has a rationale predictor head that can predict the rationales given an abusive sentence.

The dataset and models are available here: https://github.com/punyajoy/HateXplain


Binny Mathew, Punyajoy Saha, Seid Muhie Yimam, Chris Biemann, Pawan Goyal, and Animesh Mukherjee "[HateXplain: A Benchmark Dataset for Explainable Hate Speech Detection)". Accepted at AAAI 2021.


@article{mathew2020hatexplain,
  title={HateXplain: A Benchmark Dataset for Explainable Hate Speech Detection},
  author={Mathew, Binny and Saha, Punyajoy and Yimam, Seid Muhie and Biemann, Chris and Goyal, Pawan and Mukherjee, Animesh},
  journal={arXiv preprint arXiv:2012.10289},
  year={2020}
}

In [14]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification
### from models.py
from models import *
import os
import shutil

import pandas as pd
import numpy as np
import torch

from datasets import load_dataset
from torch.utils.data import DataLoader
from sklearn.preprocessing import LabelEncoder

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
torch.cuda.empty_cache()

seed = 42
batch_size = 2

In [5]:
tokenizer = AutoTokenizer.from_pretrained("Hate-speech-CNERG/bert-base-uncased-hatexplain")
model = AutoModelForSequenceClassification.from_pretrained("Hate-speech-CNERG/bert-base-uncased-hatexplain").to(device)

In [15]:
dataset = load_dataset("hatexplain", split="validation")

class_names = ["hate speech", "normal", "offensive"]
#dataset = dataset.map(lambda e: tokenizer(e['post_tokens'], truncation=True, padding='longest', is_split_into_words=True), batched=True)

data_tokens = tokenizer(dataset['post_tokens'], padding=True,is_split_into_words=True, return_tensors="pt")

#for i in range(len(dataset)):
#    print(f"data: {' '.join(dataset[i]['post_tokens'])} \n\t label: {dataset[i]['annotators']['label']}\n")


#dataloader = DataLoader(data_tokens, batch_size=batch_size, shuffle=False)

input_tensor = []
attention_mask_tensor = []
for i in range(0,len(data_tokens.input_ids), batch_size):
    input_tensor += [data_tokens.input_ids[i:i+batch_size]]
    attention_mask_tensor += [data_tokens.attention_mask[i:i+batch_size]]

Reusing dataset hatexplain (C:\Users\Ahmad\.cache\huggingface\datasets\hatexplain\plain_text\1.0.0\df474d8d8667d89ef30649bf66e9c856ad8305bef4bc147e8e31cbdf1b8e0249)


In [19]:
print(type(model))

<class 'transformers.models.bert.modeling_bert.BertForSequenceClassification'>


In [21]:
logits_list = []
for i in range(len(input_tensor)):
    with torch.no_grad():
        inp = input_tensor[i].to(device)
        attention = attention_mask_tensor[i].to(device)
        prediction_logits, what = model(input_ids=inp,attention_mask=attention)
        print(type(prediction_logits))
        print(what)

        logits_list.append(prediction_logits)
        
        inp.detach()
        attention.detach()

<class 'str'>
attentions
<class 'str'>
attentions
<class 'str'>
attentions
<class 'str'>
attentions
<class 'str'>
attentions
<class 'str'>
attentions
<class 'str'>
attentions
<class 'str'>
attentions
<class 'str'>
attentions
<class 'str'>
attentions
<class 'str'>
attentions
<class 'str'>
attentions
<class 'str'>
attentions
<class 'str'>
attentions
<class 'str'>
attentions
<class 'str'>
attentions
<class 'str'>
attentions
<class 'str'>
attentions
<class 'str'>
attentions
<class 'str'>
attentions
<class 'str'>
attentions
<class 'str'>
attentions
<class 'str'>
attentions
<class 'str'>
attentions
<class 'str'>
attentions
<class 'str'>
attentions
<class 'str'>
attentions
<class 'str'>
attentions
<class 'str'>
attentions
<class 'str'>
attentions
<class 'str'>
attentions
<class 'str'>
attentions
<class 'str'>
attentions
<class 'str'>
attentions
<class 'str'>
attentions
<class 'str'>
attentions
<class 'str'>
attentions
<class 'str'>
attentions
<class 'str'>
attentions
<class 'str'>
attentions


In [17]:
print(logits_list[:10])
print(prediction_logits)

['logits', 'logits', 'logits', 'logits', 'logits', 'logits', 'logits', 'logits', 'logits', 'logits']
logits


In [9]:
predicitons = []
for i in range(len(logits_list)):
    predicitons += [(torch.argmax(logits_list[i], dim=1).item(), dataset[i]['annotators']['label'])]

TypeError: argmax(): argument 'input' (position 1) must be Tensor, not str

In [ ]:
print(predicitons)

In [ ]:
print(f"dataset size: {len(data)}")
for i in range(len(data)):
    print(f"data: {' '.join(dataset[i]['post_tokens'])} \n\tpredictiton: {predicitons[i]} {class_names[predicitons[i]]} \n\tlable: {dataset[i]['annotators']['label']}\n")